In [1]:
# Import Dependencies
import re
import xml.etree.ElementTree as ET
import xmlschema
import pandas as pd
import yaml
from collections import OrderedDict
from ruamel.yaml import YAML

In [22]:
# File paths
#xml_input_path = r'C:/Users/jhyan/Documents/all/Research/Duke_research/MaterialsMine_YAMLconvert/resources/input_files/L183_S4_Potschke_2003_origin.xml'  # Replace with your XML file path
#yaml_output_path = r'C:/Users/jhyan/Documents/all/Research/Duke_research/MaterialsMine_YAMLconvert/resources/output_files/L183_S4_Potschke_2003.yaml'  # Replace with your output YAML file path

In [2]:
# File paths
xml_input_path = r'C:/Users/jhyan/Documents/all/Research/Duke_research/MaterialsMine_YAMLconvert/resources/input_files/L175_S5_OReilly_2015_origin.xml'  # Replace with your XML file path
yaml_output_path = r'C:/Users/jhyan/Documents/all/Research/Duke_research/MaterialsMine_YAMLconvert/resources/output_files/L175_S5_OReilly_2015.yaml'  # Replace with your output YAML file path

In [3]:

def parse_xml_to_dict(element):
    """
    Recursively parses an XML element into an OrderedDict.

    Args:
    - element (xml.etree.ElementTree.Element): The XML element to parse.

    Returns:
    - OrderedDict: OrderedDict representing the XML element.
    """
    parsed_dict = OrderedDict()

    # Process element attributes
    if element.attrib:
        parsed_dict["@attributes"] = element.attrib

    # Process child elements
    for child in element:
        child_dict = parse_xml_to_dict(child)
        if child.tag not in parsed_dict:
            parsed_dict[child.tag] = child_dict
        else:
            if not isinstance(parsed_dict[child.tag], list):
                parsed_dict[child.tag] = [parsed_dict[child.tag]]
            parsed_dict[child.tag].append(child_dict)

    # Process element text
    if element.text and element.text.strip():
        if parsed_dict:
            parsed_dict["@text"] = element.text.strip()
        else:
            return element.text.strip()

    return parsed_dict

def write_dict_to_yaml(data_dict, file_path):
    """
    Writes a dictionary to a YAML file.

    Args:
    - data_dict (dict): The dictionary to write.
    - file_path (str): The path to the YAML file.
    """
    yaml = YAML()
    yaml.default_flow_style = False
    yaml.allow_unicode = True
    yaml.explicit_start = False
    yaml.default_style = None
    
    # Convert OrderedDict to regular dict to avoid !!omap
    def convert_ordereddict(d):
        if isinstance(d, OrderedDict):
            return dict((k, convert_ordereddict(v)) for k, v in d.items())
        elif isinstance(d, list):
            return [convert_ordereddict(i) for i in d]
        else:
            return d

    clean_dict = convert_ordereddict(data_dict)

    with open(file_path, 'w', encoding='utf-8') as file:
        yaml.dump(clean_dict, file)

# Parse XML file into an OrderedDict
xml_tree = ET.parse(xml_input_path)
xml_root = xml_tree.getroot()
xml_dict = OrderedDict({xml_root.tag: parse_xml_to_dict(xml_root)})  # Ensure the root tag is included properly

# Write the OrderedDict to a YAML file without !!omap
write_dict_to_yaml(xml_dict, yaml_output_path)

print(f'YAML file has been generated and saved as {yaml_output_path}')

YAML file has been generated and saved as C:/Users/jhyan/Documents/all/Research/Duke_research/MaterialsMine_YAMLconvert/resources/output_files/L175_S5_OReilly_2015.yaml
